# Curso Agentes IA - I2A2

##Desafio - Criação de Agente com RAG

**GRUPO 294 - Integrantes**

| Nome     | E-mail | Telefone |
|----------|-----|-----|
| Larissa     | larissagarcialp.lg@gmail.com | +55 (91) 98162-3492 |
| Weslley Sbizera    | mdisolucoestecnologia@gmail.com | +55 (13) 98229-6448 |
| Felipe    | fanmello@gmail.com | +55 (31) 98420-4490 |
| Jair    | jairmilani21@gmail.com | +55 (61) 98274-5892 |
| Samuel    | fisica141@gmail.com | +55 (24) 98879-0300 |
| Isis    | isis.coelho@meta.com.br | +55 (55) 99671-8633 |
| Daniel    | montanha78@gmail.com | +55 (11) 92000-7175 |
| Eduardo    | eduardobessa10@gmail.com | +55 (11) 99509-1781 |


---

**Exercício - Entrega 18/06/2025**

A atividade tem por objetivo criar um ou mais agentes que tornem possível a um usuário realizar perguntas sobre os arquivos CSV disponibilizados.

Por exemplo: Qual é o fornecedor que teve maior montante recebido? Qual item teve maior volume entregue (em quantidade)? E assim por diante.

##Bibliotecas

In [1]:
!pip install langchain langchain-openai openai pandas langchain-experimental python-dotenv -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00


In [3]:
# @title Passo 1.2: Configuração da Chave API do OpenRouter
# --------------------------------------------------------------------------
# ATENÇÃO: Execute esta célula e insira sua chave API do OpenRouter.
# 1. Obtenha sua chave em: https://openrouter.ai/keys
# 2. Para maior segurança no Colab, use o gerenciador de "Secrets" (ícone de chave no painel esquerdo).
#    Adicione sua chave como um segredo chamado 'OPENROUTER_API_KEY'.
# --------------------------------------------------------------------------
import os
from google.colab import userdata # Para usar os Secrets do Colab
from dotenv import load_dotenv

# Tenta carregar a chave API do Secrets do Colab
try:
    os.environ['OPENROUTER_API_KEY'] = userdata.get('OPENROUTER_API_KEY')
    print("Chave API do OpenRouter carregada do Secrets do Colab!")
except userdata.SecretNotFoundError:
    print("Segredo 'OPENROUTER_API_KEY' não encontrado nos Secrets do Colab.")
    print("Por favor, insira sua chave API do OpenRouter manualmente abaixo.")
    os.environ['OPENROUTER_API_KEY'] = input("Insira sua OpenRouter API Key: ")
except Exception as e:
    print(f"Ocorreu um erro ao tentar acessar o segredo: {e}")
    print("Por favor, insira sua chave API do OpenRouter manualmente abaixo.")
    os.environ['OPENROUTER_API_KEY'] = input("Insira sua OpenRouter API Key: ")


if not os.environ.get('OPENROUTER_API_KEY'):
    print("Chave API do OpenRouter não configurada. O agente não funcionará.")
else:
    print("Chave API do OpenRouter configurada com sucesso!")

# Criar um arquivo .env simbólico para compatibilidade com load_dotenv se necessário
with open(".env", "w") as f:
    f.write(f"OPENROUTER_API_KEY={os.environ.get('OPENROUTER_API_KEY')}\n")
load_dotenv() # Carrega a variável de ambiente para o contexto atual

Chave API do OpenRouter carregada do Secrets do Colab!
Chave API do OpenRouter configurada com sucesso!


True

In [4]:
# @title Passo 2.1: Upload do Arquivo ZIP
# Execute esta célula para fazer o upload do arquivo '202401_NFs.zip'.
# Certifique-se de que o nome do arquivo é EXATAMENTE '202401_NFs.zip'.
from google.colab import files
import os

DATA_DIR = "data_colab"
ZIP_FILE_NAME = "202401_NFs.zip"
ZIP_FILE_PATH = os.path.join(DATA_DIR, ZIP_FILE_NAME)
EXTRACTED_DATA_DIR = os.path.join(DATA_DIR, "extracted_NFs")

if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

if not os.path.exists(EXTRACTED_DATA_DIR):
    os.makedirs(EXTRACTED_DATA_DIR)

# Verifica se o arquivo já existe para não pedir upload novamente desnecessariamente
if not os.path.exists(ZIP_FILE_PATH):
    print(f"Por favor, faça o upload do arquivo '{ZIP_FILE_NAME}'.")
    uploaded = files.upload()

    if ZIP_FILE_NAME in uploaded:
        with open(ZIP_FILE_PATH, 'wb') as f:
            f.write(uploaded[ZIP_FILE_NAME])
        print(f"Arquivo '{ZIP_FILE_NAME}' carregado com sucesso em '{ZIP_FILE_PATH}'")
    else:
        print(f"Erro: Arquivo '{ZIP_FILE_NAME}' não encontrado nos arquivos enviados.")
else:
    print(f"Arquivo '{ZIP_FILE_NAME}' já existe em '{ZIP_FILE_PATH}'. Pulando upload.")

Por favor, faça o upload do arquivo '202401_NFs.zip'.


Saving 202401_NFs.zip to 202401_NFs.zip
Arquivo '202401_NFs.zip' carregado com sucesso em 'data_colab/202401_NFs.zip'


In [5]:
# @title Passo 2.2: Funções de Tratamento de Arquivos (Cabeçalho Ajustado)
import pandas as pd
import zipfile
from typing import Tuple, Optional

# Nomes dos arquivos CSV dentro do ZIP
HEADER_CSV_NAME = "202401_NFs_Cabecalho.csv"
ITEMS_CSV_NAME = "202401_NFs_Itens.csv" # Assumindo este nome para o arquivo de itens

def unzip_file_colab_adj(zip_path: str, extract_to: str) -> bool:
    try:
        if not os.path.exists(extract_to): os.makedirs(extract_to)
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to)
        print(f"Arquivo '{zip_path}' descompactado em '{extract_to}'")
        print("Arquivos extraídos:", os.listdir(extract_to))
        return True
    except Exception as e:
        print(f"Erro ao descompactar '{zip_path}': {e}")
        return False

def load_csv_files_colab_adj(data_folder: str) -> Tuple[Optional[pd.DataFrame], Optional[pd.DataFrame]]:
    path_cabecalho = os.path.join(data_folder, HEADER_CSV_NAME)
    path_itens = os.path.join(data_folder, ITEMS_CSV_NAME)
    df_cabecalho, df_itens = None, None

    date_columns_cabecalho = ['DATA EMISSÃO', 'DATA/HORA EVENTO MAIS RECENTE']

    try:
        if os.path.exists(path_cabecalho):
            df_cabecalho = pd.read_csv(path_cabecalho, sep=',', decimal='.')
            for col in date_columns_cabecalho:
                if col in df_cabecalho.columns:
                    df_cabecalho[col] = pd.to_datetime(df_cabecalho[col], errors='coerce')
            print(f"Arquivo '{HEADER_CSV_NAME}' carregado. Colunas: {df_cabecalho.columns.tolist()}")
        else:
            print(f"Arquivo '{HEADER_CSV_NAME}' não encontrado em '{data_folder}'.")

        if os.path.exists(path_itens):
            df_itens = pd.read_csv(path_itens, sep=',', decimal='.')
            # Adicione aqui a conversão de colunas de data para o df_itens, se houver.
            # Ex: if 'DATA_ITEM_ALGUMA' in df_itens.columns: df_itens['DATA_ITEM_ALGUMA'] = pd.to_datetime(df_itens['DATA_ITEM_ALGUMA'], errors='coerce')
            print(f"Arquivo '{ITEMS_CSV_NAME}' carregado. Colunas: {df_itens.columns.tolist()}")
        else:
            print(f"Arquivo '{ITEMS_CSV_NAME}' não encontrado em '{data_folder}'. **A funcionalidade relacionada a itens será limitada.**")
    except Exception as e:
        print(f"Erro ao carregar arquivos CSV: {e}")
        return None, None
    return df_cabecalho, df_itens

def preprocess_data_colab_adj(df_cabecalho: pd.DataFrame, df_itens: Optional[pd.DataFrame]) -> pd.DataFrame:
    """
    Mescla os DataFrames de cabeçalho e itens.
    Se df_itens for None, retorna apenas df_cabecalho.
    """
    if df_cabecalho is None:
        print("DataFrame de cabeçalho não foi carregado. Não é possível processar.")
        return None # Ou levantar um erro

    if df_itens is None:
        print("DataFrame de itens não foi carregado ou não encontrado. Retornando apenas o DataFrame de cabeçalho.")
        print("**Perguntas sobre detalhes específicos dos itens não poderão ser respondidas.**")
        return df_cabecalho # Retorna apenas o cabeçalho se os itens não estiverem disponíveis

    # --- IMPORTANTE: Chave para o MERGE ---
    # Usando 'CHAVE DE ACESSO' como chave comum, conforme especificado.
    # **VERIFIQUE se 'CHAVE DE ACESSO' também existe no arquivo de ITENS com o mesmo significado!**
    COMMON_KEY = 'CHAVE DE ACESSO'

    if COMMON_KEY not in df_cabecalho.columns:
        print(f"Coluna chave '{COMMON_KEY}' NÃO encontrada no DataFrame de CABEÇALHO. Impossível fazer o merge.")
        print(f"Colunas do Cabeçalho: {df_cabecalho.columns.tolist()}")
        return df_cabecalho # Retorna apenas o cabeçalho para permitir perguntas sobre ele
                           # mas o merge falhou.

    if COMMON_KEY not in df_itens.columns:
        print(f"Coluna chave '{COMMON_KEY}' NÃO encontrada no DataFrame de ITENS. Impossível fazer o merge.")
        print(f"Colunas dos Itens: {df_itens.columns.tolist()}")
        print("**Retornando apenas o DataFrame de cabeçalho. Perguntas sobre itens não funcionarão corretamente.**")
        return df_cabecalho # Retorna apenas o cabeçalho

    try:
        # Realiza o merge. 'suffixes' ajuda a diferenciar colunas com mesmo nome (exceto a chave)
        df_merged = pd.merge(df_cabecalho, df_itens, on=COMMON_KEY, how='left', suffixes=('_cab', '_item'))
        print(f"DataFrames mesclados com sucesso usando a chave '{COMMON_KEY}'.")
        print(f"Colunas do DataFrame Mesclado: {df_merged.columns.tolist()}")
        return df_merged
    except Exception as e:
        print(f"Erro ao mesclar DataFrames: {e}")
        print("**Retornando apenas o DataFrame de cabeçalho devido ao erro no merge.**")
        return df_cabecalho # Em caso de erro, retorna pelo menos o cabeçalho

In [6]:
# @title Passo 2.3: Executar Descompactação, Carregamento e Pré-processamento (Cabeçalho Ajustado)
processed_df = None # Este será o DataFrame usado pelo agente (mesclado ou apenas cabeçalho)
df_cabecalho_adj_global = None
df_itens_adj_global = None

# DATA_DIR = "data_colab"
# ZIP_FILE_NAME = "202401_NFs.zip"
# ZIP_FILE_PATH = os.path.join(DATA_DIR, ZIP_FILE_NAME)
# EXTRACTED_DATA_DIR = os.path.join(DATA_DIR, "extracted_NFs")

if os.path.exists(ZIP_FILE_PATH):
    if unzip_file_colab_adj(ZIP_FILE_PATH, EXTRACTED_DATA_DIR):
        df_cabecalho_adj_global, df_itens_adj_global = load_csv_files_colab_adj(EXTRACTED_DATA_DIR)

        if df_cabecalho_adj_global is not None:
            print("\n--- Pré-processando DataFrames ---")
            # Passamos ambos os dataframes para preprocess_data_colab_adj
            # A função decidirá se o merge é possível ou se retorna apenas o cabeçalho
            processed_df = preprocess_data_colab_adj(df_cabecalho_adj_global, df_itens_adj_global)

            if processed_df is not None:
                print("\n--- DataFrame Processado (Primeiras 5 linhas) ---")
                from IPython.display import display
                display(processed_df.head())
                print(f"\nDimensões do DataFrame Processado: {processed_df.shape}")
                if df_itens_adj_global is None or 'CHAVE DE ACESSO' not in df_itens_adj_global.columns:
                     print("\nAVISO: O DataFrame de itens não foi carregado ou não contém a chave 'CHAVE DE ACESSO'.")
                     print("O DataFrame 'processed_df' contém apenas dados do cabeçalho.")
                     print("Perguntas sobre detalhes específicos dos itens não poderão ser respondidas corretamente.")
                elif not all(item in processed_df.columns for item in df_itens_adj_global.columns if item != 'CHAVE DE ACESSO'):
                     # Heurística para checar se colunas de itens (além da chave) estão no merged_df
                     # Isso pode não ser perfeito se houver sufixos. Uma checagem mais robusta seria verificar
                     # se o número de colunas aumentou significativamente após o merge.
                     if len(processed_df.columns) > len(df_cabecalho_adj_global.columns):
                         print("\nINFO: Parece que o merge com o DataFrame de itens foi realizado.")
                     else:
                         print("\nAVISO: O merge com o DataFrame de itens pode não ter adicionado novas colunas de itens como esperado.")
            else:
                print("Falha crítica no pré-processamento dos dados.")
        else:
            print("DataFrame de cabeçalho não pôde ser carregado. Verifique os logs.")
    else:
        print("Falha ao descompactar o arquivo.")
else:
    print(f"Arquivo ZIP '{ZIP_FILE_PATH}' não encontrado. Execute o Passo 2.1.")

Arquivo 'data_colab/202401_NFs.zip' descompactado em 'data_colab/extracted_NFs'
Arquivos extraídos: ['202401_NFs_Cabecalho.csv', '202401_NFs_Itens.csv']
Arquivo '202401_NFs_Cabecalho.csv' carregado. Colunas: ['CHAVE DE ACESSO', 'MODELO', 'SÉRIE', 'NÚMERO', 'NATUREZA DA OPERAÇÃO', 'DATA EMISSÃO', 'EVENTO MAIS RECENTE', 'DATA/HORA EVENTO MAIS RECENTE', 'CPF/CNPJ Emitente', 'RAZÃO SOCIAL EMITENTE', 'INSCRIÇÃO ESTADUAL EMITENTE', 'UF EMITENTE', 'MUNICÍPIO EMITENTE', 'CNPJ DESTINATÁRIO', 'NOME DESTINATÁRIO', 'UF DESTINATÁRIO', 'INDICADOR IE DESTINATÁRIO', 'DESTINO DA OPERAÇÃO', 'CONSUMIDOR FINAL', 'PRESENÇA DO COMPRADOR', 'VALOR NOTA FISCAL']
Arquivo '202401_NFs_Itens.csv' carregado. Colunas: ['CHAVE DE ACESSO', 'MODELO', 'SÉRIE', 'NÚMERO', 'NATUREZA DA OPERAÇÃO', 'DATA EMISSÃO', 'CPF/CNPJ Emitente', 'RAZÃO SOCIAL EMITENTE', 'INSCRIÇÃO ESTADUAL EMITENTE', 'UF EMITENTE', 'MUNICÍPIO EMITENTE', 'CNPJ DESTINATÁRIO', 'NOME DESTINATÁRIO', 'UF DESTINATÁRIO', 'INDICADOR IE DESTINATÁRIO', 'DESTINO D

,CHAVE DE ACESSO,MODELO_cab,SÉRIE_cab,NÚMERO_cab,NATUREZA DA OPERAÇÃO_cab,DATA EMISSÃO_cab,EVENTO MAIS RECENTE,DATA/HORA EVENTO MAIS RECENTE,CPF/CNPJ Emitente_cab,RAZÃO SOCIAL EMITENTE_cab,...,PRESENÇA DO COMPRADOR_item,NÚMERO PRODUTO,DESCRIÇÃO DO PRODUTO/SERVIÇO,CÓDIGO NCM/SH,NCM/SH (TIPO DE PRODUTO),CFOP,QUANTIDADE,UNIDADE,VALOR UNITÁRIO,VALOR TOTAL
0,41240106267630001509550010035101291224888487,55 - NF-E EMITIDA EM SUBSTITUIÇÃO AO MODELO 1 ...,1,3510129,Outras Entradas - Dev Remessa Escola,2024-01-18 07:10:39,Autorização de Uso,2024-01-18 07:10:58,06267630001509,COMPANHIA BRASILEIRA DE EDUC. E SIST. DE ENS. ...,...,"9 - OPERAÇÃO NÃO PRESENCIAL, OUTROS",1,COLECAO SPE EF1 4ANO VOL 1 AL,49019900,"Outros livros, brochuras e impressos semelhantes",2949,1.0,UNIDAD,522.5,522.5
1,50240129843878000170550010000025251000181553,55 - NF-E EMITIDA EM SUBSTITUIÇÃO AO MODELO 1 ...,1,2525,VENDA DE MERCADORIA FORA DO ESTADO,2024-01-26 11:24:42,Autorização de Uso,2024-01-26 11:24:43,29843878000170,V CALDI PEREIRA PECAS E SERVICOS DIESEL,...,1 - OPERAÇÃO PRESENCIAL,1,LANTERNA TATERAL CARRETA LED,85122021,Luzes fixas para automóveis e outros ciclos,6403,4.0,UNIDAD,39.9,159.6
2,50240129843878000170550010000025251000181553,55 - NF-E EMITIDA EM SUBSTITUIÇÃO AO MODELO 1 ...,1,2525,VENDA DE MERCADORIA FORA DO ESTADO,2024-01-26 11:24:42,Autorização de Uso,2024-01-26 11:24:43,29843878000170,V CALDI PEREIRA PECAS E SERVICOS DIESEL,...,1 - OPERAÇÃO PRESENCIAL,2,CINEMATICO RODO-AR,73071920,Acessórios moldados para tubos de aço,6403,4.0,PEÇA,75.0,300.0
3,50240129843878000170550010000025251000181553,55 - NF-E EMITIDA EM SUBSTITUIÇÃO AO MODELO 1 ...,1,2525,VENDA DE MERCADORIA FORA DO ESTADO,2024-01-26 11:24:42,Autorização de Uso,2024-01-26 11:24:43,29843878000170,V CALDI PEREIRA PECAS E SERVICOS DIESEL,...,1 - OPERAÇÃO PRESENCIAL,3,ESPIRAL NYLON CABINE AMARELO,39173100,Tubos flexíveis podendo suportar uma pressão m...,6403,1.0,PEÇA,39.9,39.9
4,50240112977901000117550010000051831659469117,55 - NF-E EMITIDA EM SUBSTITUIÇÃO AO MODELO 1 ...,1,5183,Venda de mercadorias,2024-01-22 11:01:09,Autorização de Uso,2024-01-22 10:01:09,12977901000117,MOSKO LTDA EPP,...,0 - NÃO SE APLICA,1,"AGUA MINERAL NATURAL, TIPO SEM GAS MATERIAL EM...",22011000,"Águas minerais e águas gaseificadas, não adici...",5405,9.0,GL,37.5,337.5



Dimensões do DataFrame Processado: (565, 47)

INFO: Parece que o merge com o DataFrame de itens foi realizado.


In [7]:
# @title Passo 3.1: Funções do Agente Langchain (Prompt Ajustado)
from langchain_openai import ChatOpenAI
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.agents.agent_types import AgentType

def create_csv_agent_openrouter_adj(
    df: pd.DataFrame,
    openrouter_model_name: str,
    llm_temperature: float = 0,
    verbose: bool = True
):
    openrouter_api_key = os.environ.get('OPENROUTER_API_KEY')
    if not openrouter_api_key:
        raise ValueError("A variável de ambiente OPENROUTER_API_KEY não está configurada.")

    llm = ChatOpenAI(
        model_name=openrouter_model_name,
        temperature=llm_temperature,
        openai_api_key=openrouter_api_key,
        openai_api_base="https://openrouter.ai/api/v1",
    )
    print(f"LLM configurado para usar o modelo OpenRouter: {openrouter_model_name}")

    # --- PROMPT AJUSTADO COM AS COLUNAS DO CABEÇALHO ---
    # Adicione colunas do arquivo de ITENS se o merge for bem-sucedido e você souber os nomes.
    prefix_prompt = """
    Você é um assistente de análise de dados altamente qualificado que trabalha com um DataFrame Pandas no Python.
    O DataFrame contém dados de notas fiscais. Ele pode conter dados mesclados do cabeçalho e dos itens das notas.
    Analise cuidadosamente as perguntas para determinar quais colunas do DataFrame usar.

    Todas as colunas conhecidas do CABEÇALHO e do ITENS incluem:
    'CHAVE DE ACESSO', 'MODELO_cab', 'SÉRIE_cab', 'NÚMERO_cab', 'NATUREZA DA OPERAÇÃO_cab', 'DATA EMISSÃO_cab', 'EVENTO MAIS RECENTE', 'DATA/HORA EVENTO MAIS RECENTE',
    'CPF/CNPJ Emitente_cab', 'RAZÃO SOCIAL EMITENTE_cab', 'INSCRIÇÃO ESTADUAL EMITENTE_cab', 'UF EMITENTE_cab', 'MUNICÍPIO EMITENTE_cab', 'CNPJ DESTINATÁRIO_cab',
    'NOME DESTINATÁRIO_cab', 'UF DESTINATÁRIO_cab', 'INDICADOR IE DESTINATÁRIO_cab', 'DESTINO DA OPERAÇÃO_cab', 'CONSUMIDOR FINAL_cab', 'PRESENÇA DO COMPRADOR_cab',
    'VALOR NOTA FISCAL', 'MODELO_item', 'SÉRIE_item', 'NÚMERO_item', 'NATUREZA DA OPERAÇÃO_item', 'DATA EMISSÃO_item', 'CPF/CNPJ Emitente_item', 'RAZÃO SOCIAL EMITENTE_item',
    'INSCRIÇÃO ESTADUAL EMITENTE_item', 'UF EMITENTE_item', 'MUNICÍPIO EMITENTE_item', 'CNPJ DESTINATÁRIO_item', 'NOME DESTINATÁRIO_item', 'UF DESTINATÁRIO_item', 'INDICADOR IE DESTINATÁRIO_item',
    'DESTINO DA OPERAÇÃO_item', 'CONSUMIDOR FINAL_item', 'PRESENÇA DO COMPRADOR_item', 'NÚMERO PRODUTO', 'DESCRIÇÃO DO PRODUTO/SERVIÇO', 'CÓDIGO NCM/SH', 'NCM/SH (TIPO DE PRODUTO)',
    'CFOP', 'QUANTIDADE', 'UNIDADE', 'VALOR UNITÁRIO', 'VALOR TOTAL'

    Se os dados dos ITENS foram mesclados, podem existir colunas como (adapte se souber os nomes reais dos itens):
    'NOME_PRODUTO_ITEM', 'QUANTIDADE_ITEM', 'VALOR_UNITARIO_ITEM', 'VALOR_TOTAL_ITEM', etc.
    Se a pergunta for sobre itens e o DataFrame parecer conter apenas dados do cabeçalho, informe que os detalhes dos itens não estão disponíveis.

    Seja preciso e baseie suas respostas estritamente nos dados fornecidos.
    Se uma pergunta não puder ser respondida com os dados, indique isso claramente.

    Responda em português.
    Use os nomes exatos das colunas ao se referir a elas em seus pensamentos ou na execução de código.
    """

    agent = create_pandas_dataframe_agent(
        llm,
        df,
        agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=verbose,
        prefix=prefix_prompt,
        handle_parsing_errors=True,
        allow_dangerous_code=True
    )
    return agent

def ask_agent_openrouter_adj(agent, question: str) -> str:
    try:
        response = agent.invoke({"input": question})
        return response.get("output", "Não foi possível processar a resposta a partir do output do agente.")
    except Exception as e:
        print(f"Erro ao consultar o agente: {e}")
        import traceback
        traceback.print_exc()
        return "Ocorreu um erro ao processar sua pergunta. Verifique os logs para detalhes."

In [8]:
# @title Passo 3.2: Criar o Agente (especificando o modelo OpenRouter)
csv_agent = None

if processed_df is not None:
    if os.environ.get('OPENROUTER_API_KEY'):
        # --- ESCOLHA SEU MODELO OPENROUTER AQUI ---
        # Visite https://openrouter.ai/models para ver os modelos disponíveis.
        # Copie o "Model name" completo. Exemplos:
        # "mistralai/mistral-7b-instruct" (bom e gratuito/barato)
        # "google/gemini-flash-1.5"
        # "openai/gpt-3.5-turbo" (via OpenRouter)
        # "anthropic/claude-3-haiku"
        # "google/gemma-7b-it"
        # qwen/qwen2.5-vl-72b-instruct:free
        # google/gemma-3-27b-it:free

        #chosen_openrouter_model = "qwen/qwen2.5-vl-72b-instruct:free"
        #chosen_openrouter_model = "mistralai/mistral-small-3.1-24b-instruct:free"
        #chosen_openrouter_model = "meta-llama/llama-3.3-70b-instruct:free"
        chosen_openrouter_model = "google/gemma-3-27b-it:free"
        #chosen_openrouter_model = "deepseek/deepseek-r1-zero:free"


        print(f"Tentando criar o agente com o modelo OpenRouter: {chosen_openrouter_model}")
        try:
            csv_agent = create_csv_agent_openrouter_adj(
                processed_df,
                openrouter_model_name=chosen_openrouter_model,
                verbose=True # verbose=True para ver os "pensamentos" do agente
            )
            print("Agente com OpenRouter criado com sucesso!")
        except ValueError as ve:
            print(f"Erro de valor ao criar agente: {ve}")
        except Exception as e:
            print(f"Erro inesperado ao criar agente: {e}")
            import traceback
            traceback.print_exc()
    else:
        print("Chave API do OpenRouter não configurada. Não é possível criar o agente.")
else:
    print("DataFrame mesclado (merged_df_or) não está disponível. Execute os passos anteriores.")

Tentando criar o agente com o modelo OpenRouter: google/gemma-3-27b-it:free
LLM configurado para usar o modelo OpenRouter: google/gemma-3-27b-it:free
Agente com OpenRouter criado com sucesso!


/usr/local/lib/python3.11/dist-packages/langchain_experimental/agents/agent_toolkits/pandas/base.py:283: UserWarning: Received additional kwargs {'handle_parsing_errors': True} which are no longer supported.
  warnings.warn(


In [ ]:
# @title Passo 4: Interação com o Agente via `input()` (Exemplos Ajustados)

if csv_agent:
    print("\n--- Agente Pronto para Perguntas (Cabeçalho Ajustado) ---")
    print("Use os nomes de coluna fornecidos, como 'RAZÃO SOCIAL EMITENTE', 'VALOR NOTA FISCAL', 'DATA EMISSÃO'.")
    print("Exemplos de perguntas:")
    print("  - Qual a 'RAZÃO SOCIAL EMITENTE' com a maior soma de 'VALOR NOTA FISCAL'?")
    print("  - Liste as 5 notas com maior 'VALOR NOTA FISCAL'.")
    print("  - Quantas notas fiscais foram emitidas na 'DATA EMISSÃO' '2024-01-15'?")
    print("  - Qual o 'VALOR NOTA FISCAL' total para o 'CPF/CNPJ Emitente' '12345678000199'?")
    if df_itens_adj_global is not None and 'CHAVE DE ACESSO' in df_itens_adj_global.columns and processed_df is not None and len(processed_df.columns) > len(df_cabecalho_adj_global.columns):
        print("  - (Se itens foram mesclados) Qual item (use a coluna de nome do produto do arquivo de itens) teve maior quantidade vendida (use a coluna de quantidade do arquivo de itens)?")
    else:
        print("  - (Perguntas sobre detalhes de itens podem não funcionar se o arquivo de itens não foi carregado/mesclado corretamente)")
    print("\nDigite 'sair' para terminar.")

    while True:
        try:
            user_question = input("\nSua pergunta: ")
        except EOFError:
            print("Entrada finalizada.")
            break
        if user_question.lower() == 'sair':
            print("Encerrando interação.")
            break
        if not user_question:
            print("Nenhuma pergunta inserida. Tente novamente ou digite 'sair'.")
            continue

        print("\nProcessando sua pergunta...")
        try:
            answer = ask_agent_openrouter_adj(csv_agent, user_question)
            print(f"\nResposta do Agente:\n{answer}")
        except Exception as e:
            print(f"Ocorreu um erro durante a pergunta: {e}")
else:
    print("Agente não foi inicializado. Verifique os passos anteriores e os logs de erro.")


--- Agente Pronto para Perguntas (Cabeçalho Ajustado) ---
Use os nomes de coluna fornecidos, como 'RAZÃO SOCIAL EMITENTE', 'VALOR NOTA FISCAL', 'DATA EMISSÃO'.
Exemplos de perguntas:
  - Qual a 'RAZÃO SOCIAL EMITENTE' com a maior soma de 'VALOR NOTA FISCAL'?
  - Liste as 5 notas com maior 'VALOR NOTA FISCAL'.
  - Quantas notas fiscais foram emitidas na 'DATA EMISSÃO' '2024-01-15'?
  - Qual o 'VALOR NOTA FISCAL' total para o 'CPF/CNPJ Emitente' '12345678000199'?
  - (Se itens foram mesclados) Qual item (use a coluna de nome do produto do arquivo de itens) teve maior quantidade vendida (use a coluna de quantidade do arquivo de itens)?

Digite 'sair' para terminar.

Sua pergunta: Qual a 'RAZÃO SOCIAL EMITENTE' com a maior soma de 'VALOR NOTA FISCAL'

Processando sua pergunta...


> Entering new AgentExecutor chain...

Thought: A pergunta pede para encontrar a 'RAZÃO SOCIAL EMITENTE' com a maior soma de 'VALOR NOTA FISCAL'. Para responder a isso, preciso agrupar o DataFrame pela coluna 'R